<a href="https://colab.research.google.com/github/katewall/2023class_size_predictions/blob/main/PH1700_school_prediction_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import LeaveOneOut, cross_val_score
import statsmodels.api as sm
from scipy.stats import shapiro
!pip install kaleido
import kaleido
import plotly.graph_objects as go

path = '/content/drive/MyDrive/AA: Writing/school challenge/falldf.csv'
falldf = pd.read_csv(path)
falldf

,Term,Year,PHM_1690,PH_1700,PH_1975,PH_1976,admitted.students.BaDS.MS,accepted.offer.of.admission.BaDS.MS,enrolled.students.BaDS.MS,admitted.students.BaDS.PHD,...,pre_accept_MPCH,pre_enroll_BaDS,pre_enroll_EHGES,pre_enroll_HPBS,pre_enroll_MPCH,Pct_ut,Pct_am,Grd_fem,Grd_mal,PHM_1690_raw
0,2163.0,2016,153.0,49.0,9999.0,9999.0,45.0,11.0,11.0,33.0,...,18.0,2.0,21.0,13.0,15.0,0.3753,0.3802,1750692,1221563,153.0
1,2173.0,2017,246.0,57.0,36.0,9999.0,33.0,13.0,11.0,43.0,...,18.0,5.0,19.0,9.0,16.0,0.4043,0.4016,1785060,1220055,246.0
2,2183.0,2018,207.0,92.0,36.0,9999.0,48.0,13.0,8.0,66.0,...,16.0,3.0,24.0,9.0,15.0,0.4240,0.4130,1818576,1216466,207.0
3,2193.0,2019,225.0,79.0,64.0,9999.0,42.0,15.0,12.0,72.0,...,15.0,4.0,24.0,19.0,10.0,0.4418,0.4244,1858200,1214439,225.0
4,2203.0,2020,302.0,96.0,58.0,9999.0,43.0,18.0,14.0,74.0,...,23.0,4.0,29.0,27.0,22.0,0.4807,0.4238,1922913,1219938,302.0
5,2213.0,2021,313.0,111.0,91.0,9999.0,63.0,28.0,17.0,171.0,...,19.0,4.0,43.0,15.0,17.0,0.4865,0.4447,1967393,1244038,313.0
6,2223.0,2022,217.0,108.0,50.0,9999.0,60.0,16.0,14.0,121.0,...,12.0,2.0,29.0,11.0,9.0,0.4955,0.4463,1908707,1182312,217.0
7,NaN,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.0,1.0,14.0,6.0,7.0,NaN,NaN,1893388,1158767,NaN


In [69]:
possible_predictors = ['Year', 'PH_1975', 'PH_1976',
       'admitted.students.BaDS.MS', 'accepted.offer.of.admission.BaDS.MS',
       'enrolled.students.BaDS.MS', 'admitted.students.BaDS.PHD',
       'accepted.offer.of.admission.BaDS.PHD', 'enrolled.students.BaDS.PHD',
       'admitted.students.BaDS.MPH', 'accepted.offer.of.admission.BaDS.MPH',
       'enrolled.students.BaDS.MPH', 'admitted.students.EHGES.MS',
       'accepted.offer.of.admission.EHGES.MS', 'enrolled.students.EHGES.MS',
       'admitted.students.EHGES.PHD', 'accepted.offer.of.admission.EHGES.PHD',
       'enrolled.students.EHGES.PHD', 'admitted.students.EHGES.MPH',
       'accepted.offer.of.admission.EHGES.MPH', 'enrolled.students.EHGES.MPH',
       'admitted.students.HPBS.PHD', 'accepted.offer.of.admission.HPBS.PHD',
       'enrolled.students.HPBS.PHD', 'admitted.students.HPBS.MPH',
       'accepted.offer.of.admission.HPBS.MPH', 'enrolled.students.HPBS.MPH',
       'admitted.students.MPCH.PHD', 'accepted.offer.of.admission.MPCH.PHD',
       'enrolled.students.MPCH.PHD', 'admitted.students.MPCH.MPH',
       'accepted.offer.of.admission.MPCH.MPH', 'enrolled.students.MPCH.MPH',
       'accept_BaDS', 'accept_EHGES',
       'accept_HPBS', 'accept_MPCH', 'enroll_BaDS', 'enroll_EHGES',
       'enroll_HPBS', 'enroll_MPCH', 'pre_accept_BaDS', 'pre_accept_EHGES',
       'pre_accept_HPBS', 'pre_accept_MPCH', 'pre_enroll_BaDS',
       'pre_enroll_EHGES', 'pre_enroll_HPBS', 'pre_enroll_MPCH','Pct_ut',
       'Pct_am', 'Grd_fem', 'Grd_mal']

In [70]:
possible_predictors = ['Year', 'pre_accept_BaDS', 'pre_accept_EHGES',
       'pre_accept_HPBS', 'pre_accept_MPCH', 'pre_enroll_BaDS',
       'pre_enroll_EHGES', 'pre_enroll_HPBS', 'pre_enroll_MPCH','Pct_ut',
       'Pct_am', 'Grd_fem', 'Grd_mal']

# Leave One Out CV

In [71]:
df_loo = falldf[falldf['Year'] != 2023]
y = np.array(df_loo['PH_1700']).reshape(-1,1)
loo = LeaveOneOut()
reg = linear_model.LinearRegression()
def loocv(model, df, y, possible_predictors):
  loo_dict = {}

  for i in possible_predictors:
    X = np.array(df.filter([i], axis=1)).reshape(-1,1)
    loo_dict[i] = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=loo)
  predictor_df = pd.DataFrame.from_dict(loo_dict, 'index')
  predictor_df['cv'] = -1 * predictor_df.sum(1) / df.shape[0]
  return predictor_df

predictor_df = loocv(reg, df_loo, y, possible_predictors)
predictor_df.sort_values('cv', ascending=True)[:10]

,0,1,2,3,4,5,6,cv
Pct_am,-17.907537,-199.448525,-247.511060,-155.986995,-67.212158,-9.009609,-16.207983,101.897695
Pct_ut,-13.673392,-120.884471,-438.229942,-27.371176,-77.908789,-60.218036,-8.630025,106.702262
Year,-73.960000,-94.090000,-466.935728,-42.250000,-1.816635,-65.610000,-197.871111,134.647639
Grd_fem,-100.164016,-97.338455,-525.022229,-39.786137,-74.689109,-51.009018,-144.903044,147.558859
pre_accept_EHGES,-1175.251085,-110.693565,-540.270326,-3.260834,-101.634850,-1117.286601,-321.583285,481.425792
pre_enroll_HPBS,-1628.513007,-976.562500,-284.765625,-137.807183,-15.578041,-934.056273,-1097.429849,724.958925
pre_accept_HPBS,-1625.612524,-974.312160,-333.506937,-178.554947,-35.444024,-950.694444,-1018.305473,730.918644
pre_accept_BaDS,-1894.141777,-1304.429246,-226.742491,-31.360000,-149.851367,-888.349637,-1358.296366,836.167269
pre_enroll_EHGES,-765.335952,-144.286476,-309.359017,-3.973127,-60.886864,-4220.321296,-482.951099,855.301976
pre_enroll_MPCH,-1722.069308,-1057.014844,-75.329299,-81.550551,-829.162433,-1056.992139,-1703.538296,932.236696


In [72]:
toppreds = predictor_df.sort_values('cv', ascending=True)[:10].index.tolist()
print(toppreds)
df_loo[['Pct_am','Pct_ut','Year','Grd_fem']].corr()

['Pct_am', 'Pct_ut', 'Year', 'Grd_fem', 'pre_accept_EHGES', 'pre_enroll_HPBS', 'pre_accept_HPBS', 'pre_accept_BaDS', 'pre_enroll_EHGES', 'pre_enroll_MPCH']


,Pct_am,Pct_ut,Year,Grd_fem
Pct_am,1.000000,0.955286,0.971166,0.919006
Pct_ut,0.955286,1.000000,0.983016,0.960462
Year,0.971166,0.983016,1.000000,0.924809
Grd_fem,0.919006,0.960462,0.924809,1.000000


In [73]:
X = np.array(df_loo.filter(['Pct_am','Pct_ut'], axis=1)).reshape(-1,2)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-3.738352,-156.509547,-396.455856,-110.698563,-1.57019,-26.249085,-19.654538,102.125162


In [74]:
X = np.array(df_loo.filter(['Pct_am','Year'], axis=1)).reshape(-1,2)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-1.303092,-194.639214,-444.814892,-149.5445,-51.997671,-26.750147,-129.836837,142.698051


In [75]:
X = np.array(df_loo.filter(['Pct_am','Grd_fem'], axis=1)).reshape(-1,2)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-1.174004,-155.065293,-412.836345,-120.990037,-15.010779,-5.855866,-6.059259,102.427369


In [76]:
X = np.array(df_loo.filter(['Pct_ut','Year'], axis=1)).reshape(-1,2)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-29.250616,-114.889479,-454.566014,-32.19205,-408.780891,-58.741014,-212.819123,187.319884


In [77]:
X = np.array(df_loo.filter(['Pct_ut','Grd_fem'], axis=1)).reshape(-1,2)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-20.35593,-113.560327,-480.394585,-29.918863,-83.731915,-249.074009,-1.289891,139.760789


In [78]:
X = np.array(df_loo.filter(['Year','Grd_fem'], axis=1)).reshape(-1,2)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-51.853122,-76.758523,-517.507567,-41.225096,-13.823593,-6.530025,-1080.102411,255.400048


In [79]:
X = np.array(df_loo.filter(['Pct_ut','Pct_am','Grd_fem'], axis=1)).reshape(-1,3)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-1.164714,-150.004535,-438.982185,-116.16577,-10.382266,-62.928997,-14.85144,113.497129


In [80]:
X = np.array(df_loo.filter(['Pct_ut','Pct_am','Grd_fem','Pct_am'], axis=1)).reshape(-1,4)
predictor_df = pd.DataFrame(cross_val_score(reg, X, y, scoring='neg_mean_squared_error', cv=loo).reshape(1,7), columns=np.arange(7))
predictor_df['cv'] = -1 * predictor_df.sum(1) / df_loo.shape[0]
predictor_df.sort_values('cv', ascending=True)

,0,1,2,3,4,5,6,cv
0,-1.164714,-150.004535,-438.982185,-116.16577,-10.382266,-62.928996,-14.85144,113.497129


# Model Selected

In [81]:
X = np.array(df_loo.filter(['Pct_am','Grd_fem'], axis=1))
X = sm.add_constant(X)
y = np.array(df_loo['PH_1700'])

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     15.40
Date:                Thu, 31 Aug 2023   Prob (F-statistic):             0.0132
Time:                        22:09:10   Log-Likelihood:                -24.103
No. Observations:                   7   AIC:                             54.21
Df Residuals:                       4   BIC:                             54.04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -377.2478    104.585     -3.607      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/stattools.py:74: ValueWarning:

omni_normtest is not valid with less than 8 observations; 7 samples were given.



In [82]:
trace1 = go.Scatter(
  x= df_loo['Year'],
  y= results.resid,
  fill= 'none',
  opacity= 0.6,
  type= 'scatter',
  mode= 'markers',
  name= 'Residuals'
)

fig = go.Figure()
fig.add_trace(trace1)

fig.update_layout(
  title= 'PH 1700 Residuals',
  xaxis_title='Year',
  yaxis_title='Residuals',
  legend=dict(orientation='h')
)

fig.show()

In [83]:
shapiro(results.resid)

ShapiroResult(statistic=0.8664012551307678, pvalue=0.17259185016155243)

In [84]:
results.predict()

array([ 48.59558466,  65.67874503,  76.41099079,  87.79865398,
        94.36709695, 112.22234843, 106.92658029])

# Final Prediction

In [85]:
falldf.filter(['Pct_am'], axis=1)

,Pct_am
0,0.3802
1,0.4016
2,0.4130
3,0.4244
4,0.4238
5,0.4447
6,0.4463
7,NaN


In [86]:
#Extrapolate for trend
X = np.array(df_loo.filter(['Year'], axis=1))
X = sm.add_constant(X)
y = np.array(df_loo['Pct_am'])

modelam = sm.OLS(y, X)
resultsam = modelam.fit()
print(resultsam.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.943
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                     82.97
Date:                Thu, 31 Aug 2023   Prob (F-statistic):           0.000267
Time:                        22:09:10   Log-Likelihood:                 26.911
No. Observations:                   7   AIC:                            -49.82
Df Residuals:                       5   BIC:                            -49.93
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -20.8741      2.338     -8.930      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/stattools.py:74: ValueWarning:

omni_normtest is not valid with less than 8 observations; 7 samples were given.



In [87]:
projectedam = resultsam.predict(np.array([[1,2023]]))[0]
print(f'2023 Extrapolation for A&M Graduation Rate: {projectedam.round(2)}')
grdfem = int(falldf.loc[falldf.shape[0]-1,'Grd_fem'])
print(f'2023 Projected Female Enrollment: {grdfem}')

2023 Extrapolation for A&M Graduation Rate: 0.46
2023 Projected Female Enrollment: 1893388


In [88]:
X_new = np.concatenate((np.ones((1,1)), np.array([[projectedam, grdfem]])), axis=1, dtype=float)
results.predict(X_new)
print(f'2023 Predition for PH 1700 Class Size: {results.predict(X_new)[0].round(2)}')

2023 Predition for PH 1700 Class Size: 114.69


# Plot Results

In [89]:
preds = falldf.filter(['Pct_am','Grd_fem'], axis=1)
preds.loc[7,'Pct_am'] = projectedam
preds.loc[7,'Grd_fem'] = grdfem
X_preds = np.array(preds).reshape(-1,2)
X_preds = sm.add_constant(X_preds)
y = results.predict(X_preds)

trace1 = go.Scatter(
  x= falldf['Year'],
  y= y,
  fill= 'none',
  opacity= 0.6,
  type= 'scatter',
  mode= 'lines',
  name= 'Predictions',
)

trace2 = go.Scatter(
  x= falldf['Year'],
  y= falldf['PH_1700'],
  fill= 'none',
  opacity= 0.6,
  type= 'scatter',
  mode= 'markers',
  marker = dict(size=14),
  name= 'Actuals',
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.update_layout(
  title= 'PH 1700 Class Size Predictions',
  xaxis_title='Year',
  yaxis_title='Headcount',
  legend=dict(orientation='h'))
fig.write_image('/content/fig1700_preds.png', engine='kaleido')
fig.show()

In [90]:
X_preds

array([[1.00000000e+00, 3.80200000e-01, 1.75069200e+06],
       [1.00000000e+00, 4.01600000e-01, 1.78506000e+06],
       [1.00000000e+00, 4.13000000e-01, 1.81857600e+06],
       [1.00000000e+00, 4.24400000e-01, 1.85820000e+06],
       [1.00000000e+00, 4.23800000e-01, 1.92291300e+06],
       [1.00000000e+00, 4.44700000e-01, 1.96739300e+06],
       [1.00000000e+00, 4.46300000e-01, 1.90870700e+06],
       [1.00000000e+00, 4.61328571e-01, 1.89338800e+06]])